### **Splatoon 3 side order  in depth analysis**

Side order is the latest released single player dlc for splatoon 3 on the nintendo switch. The gamemode is structured in a roadlike game, with progress being tracked by the amount of floors the player has went up.
there are a total of 30 floors after the first climb of 10 floors. Each floor will progress in difficulty, but does allow the oppurtunity for the player to enhance their weapons & abilities.

<img src="../images/side-order-main-image.webp" alt="Side order image" width="300">

These upgrades can be acquired by collecting colorchips. There are 6 categories in wich a colorchip can increase different damage, movement, mobility or even the pearl drone enhancements. The last importent feature is the Marina hacks that you can boost by (prlz) wich are the ingame currency for leveling them. Marina's hacks mostly involve player durability, lives and armor statistics.

In [109]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import ipywidgets as widgets
from plotly.subplots import make_subplots

The first thing we will look at are the damage types and their increasing scalability. They range from any source of damage that can be dealt by either the player or pearl drone. Such as main weapon, sub weapon, special weapon, pearl drone abilities and stage mechanics like the towers sprinkler.

In [2]:
powerchips = pd.read_csv('../data/colorchips & damage/colorchips/power_colorchips.csv')
rangechips = pd.read_csv('../data/colorchips & damage/colorchips/range_colorchips_main_ink.csv')
mobilitychips = pd.read_csv('../data/colorchips & damage/colorchips/mobility_colorchips.csv')
supportchips = pd.read_csv('../data/colorchips & damage/colorchips/support_colorchips.csv')
luckychips = pd.read_csv('../data/colorchips & damage/colorchips/lucky_colorchips.csv')
dronechips = pd.read_csv('../data/colorchips & damage/colorchips/drone_colorchips.csv')

exclude_columns_mchips = ['Extra Dodge Roll', 'Charge Storage', 'Squid Attack', 'Dodge Roll Attack', 'Lunge Attack', 'Mobile Special Charge', 'Mobile Drone Gauge', 'Extra comments']
mobilitychip_accepted_columns = [col for col in mobilitychips.columns[2:-1] if not any(exclude_col in col for exclude_col in exclude_columns_mchips)]

def process_colorchips(df, exclude_columns, columns):
    for col in columns:
        if col not in exclude_columns:
            if df[col].dtype == 'object' and '%' in df[col].iloc[0]:
                df[col] = df[col].str.rstrip('%').astype('float')
            else:
                df[col] = df[col].astype('float')

            return df
        
powerchip_columns = process_colorchips(powerchips, ['id'], powerchips.columns[2:-1])
rangechips_columns = process_colorchips(rangechips, ['id'], rangechips.columns[4:-2])
mobilitychip_columns = process_colorchips(mobilitychips, ['id'], mobilitychip_accepted_columns)
supportchip_columns = process_colorchips(supportchips, ['id'], supportchips.columns[2:-5])
luckychip_columns = process_colorchips(luckychips, ['id'], luckychips.columns[2:-3])
dronechip_columns = process_colorchips(dronechips, ['id'], dronechips.columns[2:])

fig = make_subplots(rows=1, cols=1)
trace_visibility = []
trace_count = 0

# Power Colorchip traces
for col in powerchips.columns[2:-1]:
    fig.add_trace(go.Scatter(
        x=powerchips['id'],
        y=powerchips[col],
        mode='lines+markers',
        name=col,
        hoverinfo='text',
        visible='legendonly',
        text=[f'Stage: {x}<br>{col}: {y}%' for x, y in zip(powerchips['id'], powerchips[col])]
    ))
    trace_visibility.append((trace_count, trace_count))
    trace_count += 1

# Range Colorchip traces
range_start = trace_count
for col in rangechips.columns[4:-2]:
    fig.add_trace(go.Scatter(
        x=rangechips['id'],
        y=rangechips[col],
        mode='lines+markers',
        name=col,
        hoverinfo='text',
        visible=False,
        text=[f'Stage: {x}<br>{col}: {y}%' for x, y in zip(rangechips['id'], rangechips[col])]
    ))
    trace_count += 1
trace_visibility.append((range_start, trace_count - 1))

# Mobility Colorchip traces
mobility_start = trace_count
for col in mobilitychip_columns:
    fig.add_trace(go.Scatter(
        x=mobilitychips['id'],
        y=mobilitychips[col],
        mode='lines+markers',
        name=col,
        hoverinfo='text',
        visible=False,
        text=[f'Stage: {x}<br>{col}: {y}%' for x, y in zip(mobilitychips['id'], mobilitychips[col])]
    ))
    trace_count += 1
trace_visibility.append((mobility_start, trace_count - 1))

# Support Colorchip traces
support_start = trace_count
for col in supportchips.columns[2:-5]:
    fig.add_trace(go.Scatter(
        x=supportchips['id'],
        y=supportchips[col],
        mode='lines+markers',
        name=col,
        hoverinfo='text',
        visible=False,
        text=[f'Stage: {x}<br>{col}: {y}%' for x, y in zip(supportchips['id'], supportchips[col])]
    ))
    trace_count += 1
trace_visibility.append((support_start, trace_count - 1))

# Lucky Colorchip traces
lucky_start = trace_count
for col in luckychips.columns[2:-3]:
    fig.add_trace(go.Scatter(
        x=luckychips['id'],
        y=luckychips[col],
        mode='lines+markers',
        name=col,
        hoverinfo='text',
        visible=False,
        text=[f'Stage: {x}<br>{col}: {y}%' for x, y in zip(luckychips['id'], luckychips[col])]
    ))
    trace_count += 1
trace_visibility.append((lucky_start, trace_count - 1))

# Drone Colorchip traces
drone_start = trace_count
for col in dronechips.columns[2:-1]:
    fig.add_trace(go.Scatter(
        x=dronechips['id'],
        y=dronechips[col],
        mode='lines+markers',
        name=col,
        hoverinfo='text',
        visible=False,
        text=[f'Stage: {x}<br>{col}: {y}p' for x, y in zip(dronechips['id'], dronechips[col])]
    ))
    trace_count += 1
trace_visibility.append((drone_start, trace_count - 1))

# create a function to shorten the code above
def create_update_args(start, end, y_title, y_ticktext):
    visibility = [False] * trace_count
    for i in range(start, end + 1):
        visibility[i] = True
    return [{'visible': visibility}, {'yaxis.title.text': y_title, 'yaxis.ticktext': y_ticktext}]

create_update_args(0, len(powerchip_columns) - 1, '<b>Increment in (%)</b>', [f"{tick}%" for tick in range(100, 1001, 100)])
create_update_args(range_start, range_start + len(rangechips_columns) - 1, '<b>Increment in (%)</b>', [f"{tick}%" for tick in range(100, 1001, 100)])
create_update_args(mobility_start, mobility_start + len(mobilitychip_columns) - 1, '<b>Increment in (%)</b>', [f"{tick}%" for tick in range(100, 1001, 100)])
create_update_args(support_start, support_start + len(supportchip_columns) - 1, '<b>Increment in (%)</b>', [f"{tick}%" for tick in range(100, 1001, 100)])
create_update_args(lucky_start, lucky_start + len(luckychip_columns) - 1, '<b>Increment in (%)</b>', [f"{tick}%" for tick in range(100, 1001, 100)])
create_update_args(drone_start, drone_start + len(dronechip_columns) - 2, '<b>Increment in (p)</b>', [f"{tick}p" for tick in range(100, 1001, 100)])


# Create the visibility and y-axis format update function
def create_update_args(start, end, y_title, y_ticktext):
    visibility = [False] * trace_count
    for i in range(start, end + 1):
        visibility[i] = True
    return [{'visible': visibility}, {'yaxis.title.text': y_title, 'yaxis.ticktext': y_ticktext}]

# Add a dropdown menu to switch between categories of colorchips
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(label='Power Colorchip', method='update',
                     args=create_update_args(0, len(powerchip_columns) - 1,
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)])),
                dict(label='Range Colorchip', method='update',
                     args=create_update_args(range_start, range_start + len(rangechips_columns) - 1,
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)])),
                dict(label='Mobility Colorchip', method='update',
                     args=create_update_args(mobility_start, mobility_start + len(mobilitychip_columns) - 1,
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)])),
                dict(label='Support Colorchip', method='update',
                     args=create_update_args(support_start, support_start + len(supportchip_columns) - 1,
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)])),
                dict(label='Lucky Colorchip', method='update',
                     args=create_update_args(lucky_start, lucky_start + len(luckychip_columns) - 1,
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)])),
                dict(label='Drone Colorchip', method='update',
                     args=create_update_args(drone_start, drone_start + len(dronechip_columns) - 2,
                                             '<b>Increment in (p)</b>',
                                             [f"{tick}p" for tick in range(100, 1001, 100)]))
            ],
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=1.285,
            xanchor='right',
            y=0.875,
            yanchor='bottom'
        )
    ]
)

# Initial layout
fig.update_layout(
    title={
        'text': '<b>Upgrades per (power) colorchip</b>',
        'y': 0.965,
        'x': 0.4,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {
            'size': 14,
            'family': 'Arial',
            'color': 'black',
        }
    },
    xaxis=dict(
        title='<b>Stage Upgrade Number</b>',
        tickmode='array',
        tickvals=list(range(1, 11)),
        ticktext=list(range(1, 11)),
        titlefont=dict(family='Arial', size=12, color='black'),
        range=[1, 10.1]
    ),
    yaxis=dict(
        title=dict(
            text='<b>Increment in (%)</b>',
            standoff=0.2  # Adjust this value to move the title to the right
        ),
        tickmode='array',
        tickvals=list(range(100, 1001, 100)),
        ticktext=[f"{tick}%" for tick in range(100, 1001, 100)],
        titlefont=dict(family='Arial', size=12, color='black')
    ),
    legend=dict(
        x=1.01,
        y=0.8,
        traceorder='normal',
        font=dict(family='Arial', size=12, color='black'),
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    hovermode='closest',
    width=900 
)

# Show the plot
fig.show()

NameError: name 'pd' is not defined